Here, I have implemented the structure of layers in class and object format of the ones mentioned in the question 1.
Then for Question 2, I have loaded the data and scaled it.
Then I have made the layers one after the other and called the training function with 5000 epochs and 0.01 as learning rate.
Then I have tested the model on same data.

In [1]:
import numpy as np

class MatrixMultiplicationLayer:
    def __init__(self, W):
        self.W = W

    def forward(self, X):
        return np.dot(X, self.W)

    def backward(self, X, grad_output):
        grad_input = np.dot(grad_output, self.W.T)
        grad_W = np.dot(X.T, grad_output)
        return grad_input, grad_W

class BiasAdditionLayer:
    def __init__(self, b):
        self.b = b

    def forward(self, X):
        return X + self.b

    def backward(self, X, grad_output):
        grad_input = grad_output
        grad_b = np.sum(grad_output, axis=0)
        return grad_input, grad_b
    
class MeanSquaredLoss:
    def forward(self, y_pred, y_true):
        return 0.5 * np.mean((y_pred - y_true) ** 2)

    def backward(self, y_pred, y_true):
        return y_pred - y_true

class Softmax:
    def forward(self, X):
        exp_X = np.exp(X)
        return exp_X / np.sum(exp_X, axis=1, keepdims=True)

    def backward(self, X, grad_output):
        softmax = self.forward(X)
        return grad_output * (softmax * (1 - softmax))

class Sigmoid:
    def forward(self, X):
        return 1 / (1 + np.exp(-X))

    def backward(self, X, grad_output):
        sigmoid = self.forward(X)
        return grad_output * sigmoid * (1 - sigmoid)

class CrossEntropyLoss:
    def forward(self, y_pred, y_true):
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def backward(self, y_pred, y_true):
        return y_pred - y_true



In [2]:
from sklearn.datasets import load_boston
boston = load_boston()
X, y = boston.data, boston.target
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

C:\Users\onlyu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        tar

In [3]:
# Initialize layers
W = np.random.randn(X.shape[1], 1)
b = np.random.randn(1)
l1 = MatrixMultiplicationLayer(W)
l2 = BiasAdditionLayer(b)
loss_layer = MeanSquaredLoss()

def train(X, y, W, b, num_epochs, learning_rate):
    for epoch in range(num_epochs):
        for i in range(X.shape[0]):
            # Forward pass
            x_i = X[i:i+1] # select ith sample
            y_i = y[i:i+1]
            z = l1.forward(x_i)
            y_pred = l2.forward(z)
            loss = loss_layer.forward(y_pred, y_i)

            # Backward pass
            grad_y_pred = loss_layer.backward(y_pred, y_i)
            grad_z, grad_b = l2.backward(z, grad_y_pred)
            grad_x, grad_W = l1.backward(x_i, grad_z)

            # Update weights
            W -= learning_rate * grad_W
            b -= learning_rate * grad_b

X = (X - X.mean(axis=0)) / X.std(axis=0)
X_test = (X - X.mean(axis=0)) / X.std(axis=0)

# Train the model
train(X, y, W, b, num_epochs=5000, learning_rate=0.01)


In [4]:
def accuracy(X, y, W, b):
    predictions = []
    for i in range(X.shape[0]):
        x_i = X[i:i+1]
        z = l1.forward(x_i)
        y_pred = l2.forward(z)
        predictions.append(y_pred)
    predictions = np.concatenate(predictions)
    accuracy = 1 - (np.abs(predictions - y) / y).mean()
    return accuracy

acc = accuracy(X, y, W, b)
print("Accuracy:", acc)

Accuracy: 0.5403766623188042
